In [1]:
!pip install ekphrasis

In [2]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [3]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "elongated", "repeated",
        'emphasis'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/home/zqxh49/anaconda3/envs/smm4h/lib/python3.8/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/home/zqxh49/anaconda3/envs/smm4h/lib/python3.8/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [4]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

def preprocess(tweet):
    tweet = ' '.join(text_processor.pre_process_doc(tweet)) # apply function normalize, segment, correct etc...
    tweet = ''.join(i for i in tweet if ord(i) < 128) # remove non-ASCII chars from data
    tweet = ' '.join(w for w in tknzr.tokenize(tweet)) # tokenize to strip of empty spaces
    return tweet

In [5]:
import pandas as pd

In [6]:
df_train = pd.read_csv('../data/interim/train.csv', header=None)
df_val = pd.read_csv('../data/interim/valid.csv', header=None)
df_test = pd.read_csv('../data/interim/test.csv', header=None)

In [7]:
# print number of lines in data before preprocessing
print(f'No of lines in train data {df_train.shape[0]}')
print(f'No of lines in train data {df_val.shape[0]}')
print(f'No of lines in train data {df_test.shape[0]}')

No of lines in train data 55419
No of lines in train data 13853
No of lines in train data 9622


In [8]:
# preprocess tweet
df_train['preprocessed_tweet'] = df_train.iloc[:,1].apply(preprocess)
df_val['preprocessed_tweet'] = df_val.iloc[:,1].apply(preprocess)
df_test['preprocessed_tweet'] = df_test.iloc[:,1].apply(preprocess)

In [9]:
# drop rows with empty preprocessed text column
df_train = df_train[df_train.preprocessed_tweet != '']
df_val = df_val[df_val.preprocessed_tweet != '']
df_test = df_test[df_test.preprocessed_tweet != '']

In [10]:
# print number of lines in data after preprocessing
print(f'No of lines in train data {df_train.shape[0]}')
print(f'No of lines in train data {df_val.shape[0]}')
print(f'No of lines in train data {df_test.shape[0]}')

No of lines in train data 55255
No of lines in train data 13816
No of lines in train data 9622


#### Export data with necessary columns only

In [12]:
df_train.to_csv('../data/processed/train.csv', columns = [0, 'preprocessed_tweet'], index=False, header=False)
df_val.to_csv('../data/processed/valid.csv', columns = [0, 'preprocessed_tweet'], index=False, header=False)
df_test.to_csv('../data/processed/test.csv', columns = [0, 'preprocessed_tweet'], index=False, header=False)